# Floodplain and terrace extraction

Notebook last updated by Simon M Mudd 09/05/2023

In this notebook we will extract some terrace and floodplains using the method described in Clubb et al, ESURF, 2017 (https://esurf.copernicus.org/articles/5/369/2017/). 

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Add some necessary packages** section. 

First we install `lsdviztools`. This will take around a minute. It is important you do this before the `condacolab` step. 

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. To get `mamba` we install something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Alternatively we can do this by downloading the mamba installer directly, but this frequently leads to various coding conflicts becasue you need to keep the installer URL up to date. `condacolab` does all that for you so you don't need to worry about it. 

In [ ]:
#%%bash
#MINICONDA_INSTALLER_SCRIPT=Mambaforge-Linux-x86_64.sh
#MINICONDA_PREFIX=/usr/local
#wget https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh &> /dev/null
#chmod +x $MINICONDA_INSTALLER_SCRIPT
#./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX &> /dev/null

Now use mamba to install `lsdtopotools`. 
This step takes a bit over a minute. 

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.8.

In [ ]:
!lsdtt-basic-metrics -v

## Add some necessary packages

First check the version of lsdviztools. **For this notebook we need lsdviztools >=0.4.9**

In [ ]:
import lsdviztools
lsdviztools.__version__

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

You can also check the version of lsdtt. This notebook was made with version *0.8*

In [ ]:
!lsdtt-basic-metrics -v

### Get the example data

The example data is an 18 Mb topographic dataset which you need to download:

In [ ]:
# Get the data from github
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChannelExtractionData/IndianCreek_1m/indian_creek.hdr", "indian_creek.hdr")
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChannelExtractionData/IndianCreek_1m/indian_creek.bil", "indian_creek.bil")

Lets make sure that file (`indian_creek.bil` and its header) have been downloaded:

In [ ]:
!ls indian_creek*

## Look at the topography

Lets start by looking at the topography:

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
### Plot the hillshade
%matplotlib inline
Base_file = "indian_creek"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")

We can get the channel sources using using `lsdtt-channel-extraction`. We will use the `print_wiener_channels` option. You also need to get the sources with the option `print_sources_to_csv`. This will take around a minute. 

In [ ]:
lsdtt_parameters = {"print_wiener_channels" : "true",
                    "print_sources_to_csv" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-channel-extraction", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Right, now that that has finished, lets see what files we got:

In [ ]:
!ls indian_creek*

There are actually 2 files with sources here, one is `indian_creek_ATsources.csv` and the other is `indian_creek_Wsources.csv`. `AT` stands for *area threshold* (and is produced by default) and `W` is for wiener. 

## Look at the Sources using folium

We can plot these sources on a folium map to compare them.
Before we do that though, we need to load the two datasets with pandas

In [ ]:
import pandas as pd

sources_AT_df = pd.read_csv("indian_creek_ATsources.csv")
sources_W_df = pd.read_csv("indian_creek_Wsources.csv")

In [ ]:
# This is for the area threshold points

import folium

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
sources_AT_df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

In [ ]:
# This is for the wiener points

import folium

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
sources_W_df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

You will notice that the Wiener sources appear at much more realistic locations on this map. If you want to use channel sources and are particular about the headwaters, you probably want to use this method (see https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2013WR015167 and https://esurf.copernicus.org/articles/4/627/2016/).

When looking for valleys and floodplains, however, you can set a minimum stream order so that the smallest channels are not captured. Even if the area threshold channel sources are not particularly accurate, the higher order streams will still be in reasonable locations. 

## Plot the points using `lsdviztools`

We can also plot these points using one of the `lsdviztools` plotting routines:

In [ ]:
sources_W_df.head()

In [ ]:
### Plot the channel sources
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "indian_creek"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "drainage_area", 
                                          scaled_data_in_log = True,
                                          points_fname = "indian_creek_Wsources.csv", 
                                          scale_points = False, manual_size =10,
                                          cmap="cubehelix", save_fig=False, size_format="geomorphology")

## Now get some floodplains

We now compute the floodplain pixels. 
This might take a little while...

In [ ]:
lsdtt_parameters = {"CHeads_file" : "indian_creek_Wsources.csv",
                    "fill_floodplain" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-valley-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Now lets see what we have:

In [ ]:
!ls indian_creek*

We can look at the resulting floodplain using the `SimpleDrape` option in `lsdviztools`.

We are going to save the image to a `png` file and then load the image since you can control the width of the image. 

In [ ]:
### Plot the valley mask
Base_file = "indian_creek"
Drape_prefix = r_prefix+"_valley"
this_FP_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="Blues", 
                                       cbar_loc = "None", colour_min_max = [0,1],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_FP_img)
from IPython.display import display, Image
display(Image(filename=this_FP_img, width=800))

## Now lets get some terraces

THIS IS CURRENTLY NOT WORKING, SMM will try to fix soon

We can now try to get some terraces. 

The terrace algorithm needs to work with a channel file, which you can get by picking sources or various otjher methods. In this example we use the valley width routine to get the longest channel and then get the terraces from there. 

In [ ]:
lsdtt_parameters = {"floodplain_raster" : "indian_creek_valley",
                    "get_valley_centreline" : "true",
                    "extract_single_channel" : "true", 
                    "channel_source_fname" : "channel_source.csv",
                    "get_valley_widths" : "true" }
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-valley-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!lsdtt-valley-metrics Test_01.driver

In [ ]:
!ls indian_creek*

Lets see where that valley centreline is. 

In [ ]:
### Plot the valley centerline
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
points_fname = "indian_creek_valley_centreline_nodes.csv"
Base_file = "indian_creek"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "drainage_area", 
                                          scaled_data_in_log = True,
                                          points_fname = "indian_creek_valley_centreline_nodes.csv", 
                                          scale_points = False, manual_size =10,
                                          cmap="cubehelix", save_fig=False, size_format="geomorphology")

In [ ]:
lsdtt_parameters = {"floodplain_raster" : "indian_creek_valley",
                    "get_terraces" : "true",
                    "use_valley_csv_for_terraces" : "true", 
                    "valley_points_csv" : "indian_creek_valley_centreline_nodes.csv" }
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-valley-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls indian_creek*

In [ ]:
### Plot some terraces 
Base_file = "indian_creek"
Drape_prefix = r_prefix+"_swath_channel_relief"
this_T_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="viridis", 
                                       cbar_loc = "None", colour_min_max = [0,100],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_T_img)
from IPython.display import display, Image
display(Image(filename=this_T_img, width=800))

## Larger scale floodplains

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "Luliang"
source_name = "COP30"

Xian_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [37.22492953703254, 110.55753854068179], 
                        upper_right_coordinates = [37.72405195691632, 111.29000638436042],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
Xian_DEM.print_parameters()
Xian_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

In [ ]:
# Import pandas library
import pandas as pd

data = [ [37.653322789440494, 111.16792841041398],
         [37.218853877510476, 110.61421760001407]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("coords_luliang.csv",index=False)
df.head()

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",
                    "get_terraces" : "true",
                    "surface_fitting_radius" : "90", 
                    "print_channels_to_csv" : "true",
                    "remove_seas" : "true",
                    "threshold_contributing_pixels" : "5000",
                    "use_absolute_thresholds" : "true",
                    "threshold_SO" : "2",
                    "fill_floodplain" : "true",
                    "extract_single_channel" : "true",
                    "channel_source_fname" : "coords_luliang.csv",
                    "relief_threshold" : "50",
                    "slope_threshold" : "0.05",
                    "minimum_terrace_height" : "5",
                    "swath_width" : "2000" }
r_prefix = "Luliang_COP30_UTM"
w_prefix = "Luliang_COP30_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-valley-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls Luliang*

In [ ]:
### Plot the topographic gradient  
Base_file = "Luliang_COP30_UTM"
Drape_prefix = Base_file+"_valley"
this_T_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="Blues", 
                                       cbar_loc = "None", colour_min_max = [0,1],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_T_img)
from IPython.display import display, Image
display(Image(filename=this_T_img, width=800))

In [ ]:
%matplotlib inline
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology",dpi=500)

In [ ]:
### Plot the topographic gradient  
Base_file = "Luliang_COP30_UTM"
Drape_prefix = "Luliang_COP30_UTM_valley"
this_FP_LL_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="Blues", 
                                       cbar_loc = "None", colour_min_max = [0,1],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_FP_LL_img)
from IPython.display import display, Image
display(Image(filename=this_FP_LL_img, width=800))

In [ ]:
### Plot the topographic gradient  
Base_file = "Luliang_COP30_UTM"
Drape_prefix = "Luliang_COP30_UTM_swath_channel_relief"
this_SCR_LL_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="viridis", 
                                       cbar_loc = "None", colour_min_max = [0,100],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_FP_LL_img)
from IPython.display import display, Image
display(Image(filename=this_FP_LL_img, width=800))

In [ ]:
!ls *terrace*

In [1]:
!lsdtt-basic-metrics -h

|| Welcome to the LSDTopoTools basic metrics tool!     ||
|| This program has a number of options for calculating||
|| simple landscape metrics.                           ||
|| This program was developed by Simon M. Mudd         ||
||  at the University of Edinburgh                     ||
|| If you use these routines please cite:              ||
|| http://doi.org/10.5281/zenodo.4577879               ||
|| If you use the roughness routine please cite:       ||
|| https://www.doi.org/10.5194/esurf-3-483-2015        ||
|| Documentation can be found at:                      ||
|| https://lsdtopotools.github.io/LSDTT_documentation/ ||
|| This is LSDTopoTools2 version                       ||
|| 0.8
|| If the version number has a d at the end it is a    ||
||  development version.                               ||
|| You have called an LSDTopoTools program.            ||
|| Prepare to explore topographic data!                ||
|| You can find some examples of usage here:           ||
|| http